In [ ]:
import duckdb

# Load SQL extension
%load_ext sql

# Initialize 🦆 DuckDB connection
conn = duckdb.connect()

# Import database
%sql conn --alias duckdb
%sql IMPORT DATABASE '../../data/nps';

Sometimes, _joins themselves_ can be incredibly useful for data transformation. Don't underestimate the power of _self-joins_ or `CROSS JOINS` using SQL.

A self-join is when we join data _to itself_ to answer a question. A `CROSS JOIN` joins _every_ record on the left to _every_ record on the right. They can be useful in certain cases, but be careful, as they can easily blow up a query. As an example, a cross-join of two 1000 record tables will produce 1,000 * 1,000 = 1,000,000 rows.

In [ ]:
%%sql
-- If I have two nights to camp at Joshua Tree,
-- how many different ways could I camp?
WITH joshua_tree_campgrounds AS (
    SELECT
        *
    FROM nps_public_data.campgrounds c
    INNER JOIN nps_public_data.parks p
        USING(parkcode)
    WHERE fullname = 'Joshua Tree National Park'
)
SELECT
    --     COUNT(*),
    --     COUNT(DISTINCT jtc.name)
    jtc.name,
    jtc2.name,
    ROW_NUMBER() OVER () as rn
FROM joshua_tree_campgrounds jtc
CROSS JOIN joshua_tree_campgrounds jtc2
WHERE jtc.name != jtc2.name
ORDER BY rn DESC

In [ ]:
%%sql
-- What about 3?
WITH joshua_tree_campgrounds AS (
    SELECT
        *
    FROM nps_public_data.campgrounds c
    INNER JOIN nps_public_data.parks p
        USING(parkcode)
    WHERE fullname = 'Joshua Tree National Park'
)
SELECT
    --     COUNT(*),
    --     COUNT(DISTINCT jtc.name)
    jtc.name,
    jtc2.name,
    jtc3.name,
    ROW_NUMBER() OVER () as rn
FROM joshua_tree_campgrounds jtc
CROSS JOIN joshua_tree_campgrounds jtc2
CROSS JOIN joshua_tree_campgrounds jtc3
WHERE 1 = 1
    AND jtc.name != jtc2.name
    AND jtc.name != jtc3.name
    AND jtc2.name != jtc3.name
ORDER BY rn desc

And we can check both: [2 nights](https://www.calculatorsoup.com/calculators/discretemathematics/permutations.php?n=9&r=2&action=solve), [3 nights](https://www.calculatorsoup.com/calculators/discretemathematics/permutations.php?n=9&r=3&action=solve)

We can also use the earlier example of unnesting states to aggregate on cumulative counts. Since our `parks` table _technically_ contains both parks and trails, we can perform a self-join on an unnested state CTE to count both parks and trails in the same query.

This answers the question "For states with a national park, how many national trails are in that state?"

In [ ]:
%%sql
WITH park_list AS (
    SELECT
        fullname,
        designation,
        UNNEST(
            SPLIT(states, ',')::string[]
            ) as state
    FROM nps_public_data.parks p
)
SELECT
    DISTINCT parks.state,
             COUNT(DISTINCT parks.fullname) as num_parks,
             COUNT(DISTINCT trails.fullname) as num_trails,
FROM park_list parks
LEFT JOIN park_list trails
    ON trails.state = parks.state
    AND trails.designation ILIKE '%trail%'
WHERE parks.designation = 'National Park'
GROUP BY 1
ORDER BY 2 DESC, 1
LIMIT 10

Pay very close attention to how and when we're filtering in the above query. The base query is selecting parks, so the filter appears in the `WHERE` clause. The self-join is designed to pull in trails, hence the filter in the `JOIN`. Finally, we're counting `DISTINCT` names to account for multiple records with the same value (duplicate rows created by the join)